# Visualizing Wildfires 01: Re-Visualizing (some of the) charts and maps from my undergrad thesis 

De haber descubierto Altair mucho antes de empezar la tesis, creo que hubiese hecho una página de internet con visualizaciones interactivas. Sin embargo, lo descubrí algo _tarde_ pero eso no significa que no pueda replicar o re-visualizar algunas cosas.

En esta _jupyter notebook_ intentaré realizar un mapa estático y otro interactivo de los incendios forestales en todo México.

In [1]:
# Data manipulation
import numpy as np
import pandas as pd
import geopandas

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt

# Report in DataPane
import datapane as dp

## Load the data

Los datos son del script [**`data_prep.py`**]()

In [2]:
# Instances (wildfire records)
gdf_wildfires_points_all = geopandas.read_file("./../../data/vector_conafor_wildfires_2017_points_clean_2022-03-02.geojson")
# State info
gdf_mx_states_all = geopandas.read_file("./../../data/vector_conafor_wildfires_2017_states_clean_2022-03-02.geojson")

## Relevant features

La información para las visualizaciones es _poca_ y es la siguiente:

In [3]:
list_relevant_vars_points = ['Latitude','Longitude','Estado', 'Duración días',
                             'Tamaño', 'Total', 'Fecha Inicio', 'ANP hectareas', 'geometry']
gdf_wildfires_points = gdf_wildfires_points_all[list_relevant_vars_points]

## Translate values

La publicación de las visualizaciones serán en inglés y por ello vamos a traducir tanto el nombre de las columnas asi como los valores de las categorías

In [4]:
gdf_wildfires_points = gdf_wildfires_points.rename(columns={'Estado':'State','Duración días':'DurationDays',
                       'Tamaño':'TypeSize','Total':'TotalHectares', 'Fecha Inicio':'Date',
                       'ANP hectareas':'HectaresANP'})

gdf_wildfires_points['DurationDays'] = gdf_wildfires_points['DurationDays'].astype('category')\
                                       .cat.rename_categories(['1 day','2 to 3 days','4 to 7 days','More than 7 days'])\
                                       .astype('object')

gdf_wildfires_points['TypeSize'] = gdf_wildfires_points['TypeSize'].astype('category')\
                                   .cat.rename_categories(['0 to 5 hectares', '11 to 20 hectares',
                                                           '21 to 50 hectares','51 to 100 hectares',
                                                           '6 to 10 hectares', 'More than 100 hectares'])\
                                   .astype('object')

gdf_mx_states_all = gdf_mx_states_all.rename(columns={'Estado':'State', 'ANP hectareas':'HectaresANP',
                                              'Total':'TotalHectares', 'NumIncendios':'NumWildfires' })

## Get month of the year

In [5]:
gdf_wildfires_points['Date'] = pd.to_datetime(gdf_wildfires_points['Date'])
gdf_wildfires_points['Month'] = gdf_wildfires_points['Date'].dt.month_name()#.astype('category')

## Static Map (State geometries + Day duration + Affected Area)

***
**CONAFOR** (National Forest Commission) is responsible for all the programs and management of forest and vegetation in Mexico, including wildfire records.

The available records I could get from Mexico's open data platform were from 2017.

Here, I'm presenting a static and interactive map that showcases Mexico's situation throughout that year.
***

In [6]:
# Base Map
base = alt.Chart(data=gdf_mx_states_all).mark_geoshape(fill='transparent', stroke='black', strokeWidth=0.2)

caption_p1 = alt.Chart({'values':[{}]}).mark_text(align='left', font='Georgia', fontStyle='italic', fontSize=10)\
.encode(x=alt.value(50),y=alt.value(330),text=alt.value(['Data: Comisión Nacial Forestal (CONAFOR)',
                                                         'Visualization by Isaac Arroyo (@unisaacarroyo)']))

points = alt.Chart(gdf_wildfires_points.sample(n=5000)).mark_circle(opacity=0.4, stroke='black', strokeWidth=0.2)\
         .encode(latitude = alt.Latitude(shorthand='Latitude'),
                 longitude = alt.Longitude(shorthand='Longitude'),
                 color = alt.Color(shorthand='DurationDays:N',
                                   scale = alt.Scale(range=['#8d1c06','#e67424','#f5c34d','#ed9b49',]),
                                   legend = alt.Legend(title='Duration days', orient='none', legendX=475,
                                                       legendY=120, symbolStrokeWidth=0.5)),
                 size = alt.Size(shorthand='TypeSize:O',
                                 legend = alt.Legend(title='Wildfire size', orient='none', legendX=475,
                                                     legendY=5, symbolStrokeWidth=0.5),
                                 sort=['0 to 5 hectares', '6 to 10 hectares',
                                       '11 to 20 hectares','21 to 50 hectares',
                                       '51 to 100 hectares','More than 100 hectares'],
                                 scale = alt.Scale(range=[10,100])),
                 tooltip = [alt.Tooltip(shorthand='State', title='State'),
                            alt.Tooltip(shorthand='DurationDays', title='Duration days'),
                            alt.Tooltip(shorthand='TotalHectares', title='Affected area (ha)'),
                            alt.Tooltip(shorthand='Month', title='When?')]
                )
p1 = base + points + caption_p1
p1 = p1\
.properties(width=500, height=350,
              title=alt.TitleParams(text=['Map of wildfire records.',' Mexico, 2017'],
                                    orient="none", align='left',dx=50, dy=280,
                                    fontSize=20 
                                   )
           )\
.configure(font='Georgia', background='white')\
.configure_view(stroke=None)\
.configure_legend(fillColor = 'transparent', padding= 5, cornerRadius=10,
                  labelFontSize = 10.5, titleFontSize = 12.5)

p1

alt.LayerChart(...)

## Interactive Map (Day duration + Affected Area)

In [7]:
# Interactive Map
points_interactive = alt.Chart(gdf_wildfires_points.sample(n=5000))\
                     .mark_circle(opacity=0.6, stroke='black', strokeWidth=0.2)\
                     .encode(x = alt.X(shorthand='Longitude', scale=alt.Scale(domain=[-120,-85]),axis=alt.Axis(title='')),
                             y = alt.Y(shorthand='Latitude', scale=alt.Scale(domain=[14,34]),axis=alt.Axis(title='')),
                             color = alt.Color(shorthand='DurationDays:N',
                                               scale = alt.Scale(range=['#8d1c06','#e67424','#f5c34d','#ed9b49',]),
                                               legend = alt.Legend(title='Duration days', orient='none', legendX=475,
                                                                   legendY=120, symbolStrokeWidth=0.5)),
                             size = alt.Size(shorthand='TypeSize:O',
                                             legend = alt.Legend(title='Wildfire size', orient='none', legendX=475,
                                                                 legendY=0, symbolStrokeWidth=0.5),
                                             sort=['0 to 5 hectares', '6 to 10 hectares','11 to 20 hectares',
                                                   '21 to 50 hectares','51 to 100 hectares','More than 100 hectares'],
                                             scale = alt.Scale(range=[10,100])),
                             tooltip = [alt.Tooltip(shorthand='State', title='State'),
                                        alt.Tooltip(shorthand='DurationDays', title='Duration days'),
                                        alt.Tooltip(shorthand='TotalHectares', title='Affected area (ha)'),
                                        alt.Tooltip(shorthand='Month', title='When?')]
                            )
p1_interactive = points_interactive + caption_p1
p1_interactive=  p1_interactive\
.properties(width=500, height=350,
              title=alt.TitleParams(text=['Interactive map of','wildfire records.','Mexico, 2017'],
                                    orient="none", align='left',dx=50, dy=245,
                                    fontSize=20))\
.configure(font='Georgia')\
.configure_view(stroke=None)\
.configure_legend(fillColor = '#FBFBFB', padding= 5, cornerRadius=10,
                  labelFontSize = 10.5, titleFontSize = 12.5)\
.configure_axis(grid=False, ticks=False, labels=False, domain=False)\
.interactive()

p1_interactive

alt.LayerChart(...)

## Report in Datapane

In [8]:
md_description = """
# Visualizing Wildfires 01: Re-Visualizing (some of the) charts and maps from my [undergrad thesis](https://github.com/isaacarroyov/thesis_undergrad)

**CONAFOR** (National Forest Commission) is responsible for all the programs and management of forest and vegetation in Mexico, including wildfire records.

The available records I could get from Mexico's open data platform were from 2017.

Here, I'm presenting a static and interactive map that showcases Mexico's situation throughout that year.
"""

In [9]:
dp.Report(
    md_description,
    dp.Select(blocks=[
        dp.Plot(p1, label="Static map (State geometries + Day duration + Affected Area)"),
        dp.Plot(p1_interactive, label = "Interactive map (Day duration + Affected Area")
    ])
).upload(name="Visualizing Wildfires 01", visibility= dp.Visibility.PORTFOLIO)

Uploading report and associated data - *please wait...*

Your report only contains a single element - did you know you can include additional plots, tables and text in a single report? More info <a href='https://docs.datapane.com/reports/blocks/layout-pages-and-selects' target='_blank'>here</a>

Report successfully uploaded. View and share your report <a href='https://datapane.com/u/unisaacarroyov/reports/VkBgrZA/visualizing-wildfires-01/' target='_blank'>here</a>, or edit your report <a href='https://datapane.com/u/unisaacarroyov/reports/VkBgrZA/visualizing-wildfires-01/edit/' target='_blank'>here</a>.

## Prepare data for R

---

Esta notebook es complemento de la visualización creada con R (que puedes ver [aquí](https://github.com/isaacarroyov/data_visualization_practice/tree/master/R))

----
Para hacer la visualización en R decidí manipular los datos con geopandas y dejarlos listos solo para visualizarlos con R

In [10]:
gdf_mx_states_all['PercentageANPHectares'] = (gdf_mx_states_all['HectaresANP']/gdf_mx_states_all['TotalHectares']) * 100

In [11]:
fourth_column = gdf_mx_states_all.pop("PercentageANPHectares")
gdf_mx_states_all.insert(4, "PercentageANPHectares", fourth_column)

In [12]:
from datetime import date
today_date = date.today().strftime("%Y-%m-%d")

gdf_mx_states_all.to_file(f"./../../data/R_vector_conafor_wildfires_2017_states_clean_en_{today_date}.geojson",
                          driver='GeoJSON')